In [1]:
import sys; sys.path.append('../..')
import random
import pyzx as zx
import os
import pickle
import time

### Heuristic simplification
- When simplifying ZX-diagrams with T-spiders, simplification routines like full_reduce lead to a very high two-qubit gate count.
- When using heuristic-based approaches we can circumvent the problem to some extent leading to better overall circuit cost after optimization

In [2]:
random.seed(1344)
g = zx.generate.cliffordT(qubits=8, depth=500, p_t=0.3)
c = zx.Circuit.from_graph(g)
print(c.stats())

Circuit  on 8 qubits with 500 gates.
        155 is the T-count
        345 Cliffords among which 
        114 2-qubit gates (114 CNOT, 0 other) and
        0 Hadamard gates.


In [3]:
c = zx.optimize.basic_optimization(c.split_phase_gates())
g = c.to_graph()
g_tele = zx.simplify.teleport_reduce(g)
g_tele.track_phases = False
zx.Circuit.from_graph(g).split_phase_gates().stats()

'Circuit  on 8 qubits with 349 gates.\n        69 is the T-count\n        280 Cliffords among which \n        114 2-qubit gates (107 CNOT, 7 other) and\n        73 Hadamard gates.'

In [4]:
g_full = g_tele.copy()
zx.simplify.full_reduce(g_full)
print(zx.extract_circuit(g_full.copy()).stats())
g_full.normalize()
zx.draw(g_full)

Circuit  on 8 qubits with 381 gates.
        69 is the T-count
        312 Cliffords among which 
        214 2-qubit gates (34 CNOT, 180 other) and
        93 Hadamard gates.


In [7]:
g_greedy = g_tele.copy()
zx.simplify.greedy_simp(g_greedy)
print(zx.extract_circuit(g_greedy.copy()).stats())
g_greedy.normalize()
zx.draw(g_greedy)

Circuit  on 8 qubits with 368 gates.
        69 is the T-count
        299 Cliffords among which 
        106 2-qubit gates (2 CNOT, 104 other) and
        153 Hadamard gates.


In [10]:
g_nu = g_tele.copy()
zx.simplify.greedy_simp_neighbors(g_nu)
print(zx.extract_circuit(g_nu.copy()).stats())
g_nu.normalize()
zx.draw(g_nu)

Circuit  on 8 qubits with 369 gates.
        69 is the T-count
        300 Cliffords among which 
        105 2-qubit gates (2 CNOT, 103 other) and
        155 Hadamard gates.


In [6]:
g_nu_sim = g_tele.copy()
g_nu_sim = zx.simplify.sim_anneal_simp_neighbors(g_nu_sim, iterations=10, alpha=0.995, cap=-2)



pos_neg ratio 90 251
best eval  373.0 curr  373.0 match  1.0
pos_neg ratio 91 245
best eval  371.0 curr  371.0 match  2
pos_neg ratio 88 244
best eval  370.0 curr  370.0 match  1
pos_neg ratio 85 244
best eval  370.0 curr  372.0 match  -2.0
pos_neg ratio 86 245
best eval  367.0 curr  367.0 match  5
pos_neg ratio 83 251
best eval  367.0 curr  367.0 match  0.0
pos_neg ratio 82 245
best eval  367.0 curr  368.0 match  -1
pos_neg ratio 84 240
best eval  367.0 curr  369.0 match  -1
pos_neg ratio 84 241
best eval  367.0 curr  368.0 match  1.0
pos_neg ratio 82 242
best eval  367.0 curr  367.0 match  1.0
pos_neg ratio 81 241
best eval  367.0 curr  367.0 match  0
pos_neg ratio 81 241
best eval  367.0 curr  369.0 match  -2
pos_neg ratio 83 248
best eval  367.0 curr  369.0 match  0.0
pos_neg ratio 83 249
best eval  367.0 curr  368.0 match  1.0
pos_neg ratio 82 248
best eval  367.0 curr  367.0 match  1.0
pos_neg ratio 83 245
best eval  366.0 curr  366.0 match  1.0
pos_neg ratio 82 245
best eval  36

best eval  353.0 curr  360.0 match  -1
pos_neg ratio 37 180
best eval  353.0 curr  358.0 match  2
pos_neg ratio 35 175
best eval  353.0 curr  357.0 match  1.0
pos_neg ratio 34 174
best eval  353.0 curr  359.0 match  -2.0
pos_neg ratio 36 174
best eval  353.0 curr  358.0 match  1
pos_neg ratio 35 175
best eval  353.0 curr  357.0 match  1
pos_neg ratio 34 169
best eval  353.0 curr  353.0 match  4
pos_neg ratio 33 166
best eval  353.0 curr  355.0 match  -2
pos_neg ratio 35 168
best eval  353.0 curr  355.0 match  0
pos_neg ratio 35 168
best eval  353.0 curr  354.0 match  1.0
pos_neg ratio 34 167
best eval  353.0 curr  354.0 match  0
pos_neg ratio 34 167
best eval  353.0 curr  354.0 match  0
pos_neg ratio 34 167
best eval  353.0 curr  354.0 match  0
pos_neg ratio 34 167
best eval  353.0 curr  353.0 match  1.0
pos_neg ratio 33 165
best eval  353.0 curr  355.0 match  -2
pos_neg ratio 34 163
best eval  353.0 curr  354.0 match  1.0
pos_neg ratio 32 158
best eval  353.0 curr  356.0 match  -2
pos

best eval  342.0 curr  350.0 match  -1
pos_neg ratio 15 144
best eval  342.0 curr  350.0 match  0
pos_neg ratio 16 142
best eval  342.0 curr  349.0 match  1.0
pos_neg ratio 15 144
best eval  342.0 curr  348.0 match  1.0
pos_neg ratio 14 144
pos_neg ratio 14 144
pos_neg ratio 14 144
best eval  342.0 curr  348.0 match  0
pos_neg ratio 14 144
best eval  342.0 curr  346.0 match  2.0
pos_neg ratio 12 142
best eval  342.0 curr  348.0 match  -2.0
pos_neg ratio 14 141
best eval  342.0 curr  344.0 match  4
pos_neg ratio 13 144
best eval  342.0 curr  344.0 match  0
pos_neg ratio 13 138
best eval  342.0 curr  343.0 match  1
pos_neg ratio 11 135
best eval  341.0 curr  341.0 match  2
pos_neg ratio 10 130
best eval  340.0 curr  340.0 match  1.0
pos_neg ratio 9 127
best eval  339.0 curr  339.0 match  1.0
pos_neg ratio 9 125
best eval  338.0 curr  338.0 match  1
pos_neg ratio 8 134
best eval  337.0 curr  337.0 match  1.0
pos_neg ratio 7 134
best eval  337.0 curr  337.0 match  0.0
pos_neg ratio 7 130
p

In [ ]:
print(zx.extract_circuit(g_nu_sim.copy()).stats())

In [5]:
g_norm = g_nu_sim.copy()
g_norm.normalize()
zx.draw(g_norm)

In [9]:
g_norm = g_nu_sim.copy()
g_norm.normalize()

In [10]:
zx.draw(g_norm)

In [12]:
zx.compare_tensors(zx.extract_circuit(g_nu_sim.copy()),c)

True